# Orchestrating Script on Multiple Virtual Machines

Learning how to run a script on a bunch of virtual machines programatically from a central controller machine. Hypothetically I could use this to divide work for a complex task among multiple machines. I'll be using ssh to communicate with the machines

In [ ]:
import paramiko
import time
import codecs
import os
import stat
from random import randint
import concurrent.futures

# Here would be the ip addresses for our workers
workers = [

]

bufsize = int(2**16)

Connect to server and execute some commands in an interactive shell

In [ ]:
commands = [
    'cd workerdir',
    'ls -a']

# Run commands in ssh client
client = paramiko.client.SSHClient()
client.load_system_host_keys()
client.connect(workers[0], username='worker')
shell = client.invoke_shell()
time.sleep(0.1)
banner_out = shell.recv(bufsize)
banner_out = codecs.decode(banner_out, 'utf-8')
output = banner_out
for command in commands:
    command_enc = codecs.encode(command + '\n', 'utf-8')
    shell.send(command_enc)
    time.sleep(0.1)
    command_out = shell.recv(bufsize)
    command_out = codecs.decode(command_out, 'utf-8')
    output += command_out
client.close()

# Get output
print(output)

Transfer a script to the worker

In [ ]:
client = paramiko.client.SSHClient()
client.load_system_host_keys()
client.connect(workers[0], username='worker')
sftp = client.open_sftp()
time.sleep(0.1)
stats = sftp.put('worker-script.py', 'workerdir/worker-script.py')
client.close()

Run script on worker

In [ ]:
# Random nth prime number to find
n = randint(2100, 2800)

# Run on worker
client = paramiko.client.SSHClient()
client.load_system_host_keys()
client.connect(workers[0], username='worker')
stdin, stdout, stderr = client.exec_command(f'python workerdir/worker-script.py {n} > workerdir/result.txt')
stdout.channel.recv_exit_status()
sftp = client.open_sftp()
with sftp.open('workerdir/result.txt', 'r') as f:
    data = f.read()
    data = codecs.decode(data, 'utf-8')
client.close()
number = int(data)

# Return number
print(number)

Now do this concurrently

In [ ]:
# Create array of nth prime numbers to find
nth_primes = [
    randint(2100, 3200)
    for _ in range(9)
]

# Partition array
nth_prime_partition = []
for i, worker in enumerate(workers):
    s = i*len(nth_primes)//len(workers)
    e = (i + 1)*len(nth_primes)//len(workers)
    nth_prime_partition.append((worker, nth_primes[s:e]))

def generate_prime_number(worker, nth_primes):
    """
    Generate prime numbers for worker
    
    :worker:     ip address of worker
    :nth_primes: list of n'th prime numbers to find
    """
    client = paramiko.client.SSHClient()
    client.load_system_host_keys()
    client.connect(worker, username='worker')
    stdin, stdout, stderr = client.exec_command('rm -f workerdir/result.txt')
    stdout.channel.recv_exit_status()
    for n in nth_primes:
        stdin, stdout, stderr = client.exec_command(f'python workerdir/worker-script.py {n} >> workerdir/result.txt')
        stdout.channel.recv_exit_status()
    sftp = client.open_sftp()
    with sftp.open('workerdir/result.txt', 'r') as f:
        data = f.read()
        data = codecs.decode(data, 'utf-8')
    client.close()
    return [ int(num) for num in data.split('\n') if num ]


# Run commands on workers using separate threads
with concurrent.futures.ThreadPoolExecutor(max_workers=len(workers)) as exe:
    futures_with_workers = { 
        exe.submit(generate_prime_number, worker, nth_primes) : (worker, nth_primes) 
        for worker, nth_primes in nth_prime_partition }
    for future in concurrent.futures.as_completed(futures_with_workers):
        worker, nth_primes = futures_with_workers[future]
        try:
            primes = future.result()
            print('Worker', worker, 'generated:')
            for nth, prime in zip(nth_primes, primes):
                print(f'\t{nth} = {prime}')
        except Exception as exc:
            print('Worker', worker, 'resulted in error:', exc)